<a href="https://colab.research.google.com/github/branjbar/nlp-bert-family/blob/master/pytorch/pythorch_roberta_gpu_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# To avoid notebook get disconnected
"""
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button").click() 
}setInterval(ClickConnect,60000)
"""

"""
caffeinate
"""

'\ncaffeinate\n'

In [0]:
# TODOS:
#X add tic and toc to calculate the total time!
## check the pretrained model
## remove neutral from the filtering
## check the configuraion values here: https://www.youtube.com/watch?v=2oWf4v6QEV8

In [0]:
# import os
# assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

# VERSION = "20200325"  # @ param ["1.5" , "20200325", "nightly"]
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version $VERSION

In [4]:
from google.colab import drive
drive.mount('/content/drive')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Thu Apr 30 10:01:28 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================

In [0]:
# import torch
# import torch_xla
# import torch_xla.core.xla_model as xm
# import torch_xla.distributed.xla_multiprocessing as xmp

In [6]:
!pip install transformers
import numpy as np
import pandas as pd
import os
import tokenizers
import string
import torch
import transformers
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm
import re
from sklearn import model_selection
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
import time

     |████████████████████████████████| 573kB 4.7MB/s 
     |████████████████████████████████| 890kB 52.5MB/s 
     |████████████████████████████████| 1.0MB 51.7MB/s 
     |████████████████████████████████| 3.7MB 45.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=15f7c20e4786a8a9a9bb75d5b15bd6c5ff3625874885588fbaab05f405f2903b
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [0]:
KAGGLE_INPUT_PATH = "../input/"
COLAB_INPUT_PATH = "/content/drive/My Drive/input/"
INPUT_PATH = COLAB_INPUT_PATH

DATA_PATH = INPUT_PATH + "tweet-sentiment-extraction/"
CHECKPOINT_PATH = INPUT_PATH + "model_checkpoint/roberta/"
CHECKPOINT_PATH_TORCH = INPUT_PATH + "model_checkpoint/roberta/torch/"
# ROBERTA_PATH = INPUT_PATH + 'models/output_twitter/'
ROBERTA_PATH = INPUT_PATH + 'models/roberta-base/'

VER='tpu-v1'

In [0]:
def print_to_file(text):
  try:
    f = open(CHECKPOINT_PATH_TORCH + VER + "-logs.txt", 'a')
  except:
    f = open(CHECKPOINT_PATH_TORCH + VER + "-logs.txt", 'w')

  print(text)
  f.write(time.strftime("%Y%m%d-%H%M%S"))
  f.write("  -- ")
  f.write(text)
  f.write("\n")

class Timer(object):
    def __init__(self, name=None):
        self.name = name

    def __enter__(self):
        self.tstart = time.time()

    def __exit__(self, type, value, traceback):
        if self.name:
            print_to_file('[%s]' % self.name,)
        print_to_file('Elapsed: %.2fmin' % ((time.time() - self.tstart)/60))

def jaccard(str1, str2): 

    a = set(str(str1).lower().split()) 
    b = set(str(str2).lower().split())

    c = a.intersection(b)

    if (len(a) + len(b) - len(c)) > 0:
      return float(len(c)) / (len(a) + len(b) - len(c))
    else:
      return 0

In [0]:
########## CONFIG #####################
class config():
    """
    basic configuration of the model/training
    """
    MAX_LEN = 192
    TRAIN_BATCH_SIZE = 32
    VALID_BATCH_SIZE = 8
    EPOCHS = 3
    DEVICE = torch.device("cuda")
    # DEVICE = xm.xla_device()
    TOKENIZER = tokenizers.ByteLevelBPETokenizer(
        vocab_file=f"{ROBERTA_PATH}/vocab.json", 
        merges_file=f"{ROBERTA_PATH}/merges.txt", 
        lowercase=True,
        add_prefix_space=True
        )

In [0]:
class TweetModel(transformers.BertPreTrainedModel):
    def __init__(self, conf):
        super(TweetModel, self).__init__(conf)
        self.roberta = transformers.RobertaModel.from_pretrained(ROBERTA_PATH, config=conf)
        self.drop_out = nn.Dropout(0.1)
        self.l0 = nn.Linear(768 * 2, 2)
        torch.nn.init.normal_(self.l0.weight, std=0.02)
    
    def forward(self, ids, mask, token_type_ids):
        _, _, out = self.roberta(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids
        )

        out = torch.cat((out[-1], out[-2]), dim=-1)
        out = self.drop_out(out)
        logits = self.l0(out)

        start_logits, end_logits = logits.split(1, dim=-1)

        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        return start_logits, end_logits

In [0]:
def process_data(tweet, selected_text, sentiment, tokenizer, max_len):
    tweet = " " + " ".join(str(tweet).split())
    selected_text = " " + " ".join(str(selected_text).split())

    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None

    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind+len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break

    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    tok_tweet = tokenizer.encode(tweet)
    input_ids_orig = tok_tweet.ids
    tweet_offsets = tok_tweet.offsets
    
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]

    sentiment_id = {
        'positive': 1313,
        'negative': 2430,
        'neutral': 7974
    }
    
    input_ids = [0] + [sentiment_id[sentiment]] + [2] + [2] + input_ids_orig + [2]
    token_type_ids = [0, 0, 0, 0] + [0] * (len(input_ids_orig) + 1)
    mask = [1] * len(token_type_ids)
    tweet_offsets = [(0, 0)] * 4 + tweet_offsets + [(0, 0)]
    targets_start += 4
    targets_end += 4

    padding_length = max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([1] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        tweet_offsets = tweet_offsets + ([(0, 0)] * padding_length)
    
    return {
        'ids': input_ids,
        'mask': mask,
        'token_type_ids': token_type_ids,
        'targets_start': targets_start,
        'targets_end': targets_end,
        'orig_tweet': tweet,
        'orig_selected': selected_text,
        'sentiment': sentiment,
        'offsets': tweet_offsets
    }


class TweetDataset:
    def __init__(self, tweet, sentiment, selected_text):
        self.tweet = tweet
        self.sentiment = sentiment
        self.selected_text = selected_text
        self.tokenizer = config.TOKENIZER
        self.max_len = config.MAX_LEN
    
    def __len__(self):
        return len(self.tweet)

    def __getitem__(self, item):
        data = process_data(
            self.tweet[item], 
            self.selected_text[item], 
            self.sentiment[item],
            self.tokenizer,
            self.max_len
        )

        return {
            'ids': torch.tensor(data["ids"], dtype=torch.long),
            'mask': torch.tensor(data["mask"], dtype=torch.long),
            'token_type_ids': torch.tensor(data["token_type_ids"], dtype=torch.long),
            'targets_start': torch.tensor(data["targets_start"], dtype=torch.long),
            'targets_end': torch.tensor(data["targets_end"], dtype=torch.long),
            'orig_tweet': data["orig_tweet"],
            'orig_selected': data["orig_selected"],
            'sentiment': data["sentiment"],
            'offsets': torch.tensor(data["offsets"], dtype=torch.long)
        }

In [0]:
def calculate_jaccard_score(
    original_tweet, 
    target_string, 
    sentiment_val, 
    idx_start, 
    idx_end, 
    offsets,
    verbose=False):
    
    if idx_end < idx_start:
        idx_end = idx_start
    
    filtered_output  = ""
    for ix in range(idx_start, idx_end + 1):
        filtered_output += original_tweet[offsets[ix][0]: offsets[ix][1]]
        if (ix+1) < len(offsets) and offsets[ix][1] < offsets[ix+1][0]:
            filtered_output += " "

    if len(original_tweet.split()) < 2:
        filtered_output = original_tweet

    if sentiment_val != "neutral" and verbose == True:
        if filtered_output.strip().lower() != target_string.strip().lower():
            print("********************************")
            print(f"Output= {filtered_output.strip()}")
            print(f"Target= {target_string.strip()}")
            print(f"Tweet= {original_tweet.strip()}")
            print("********************************")

    jac = jaccard(target_string.strip(), filtered_output.strip())
    return jac, filtered_output

In [0]:
train_df = pd.read_csv(DATA_PATH + 'train.csv').dropna().reset_index()#.sample(n=2000).reset_index()
df_test = pd.read_csv(DATA_PATH + 'test.csv')
df_test.loc[:, "selected_text"] = df_test.text.values

In [0]:
model_config = transformers.RobertaConfig.from_pretrained(ROBERTA_PATH)
model_config.output_hidden_states = True

In [16]:
# %%capture

model_list = []
for i in range(5):
    print("loading model %i"%i)
    model = TweetModel(conf=model_config)
    model.to(config.DEVICE)
    model.load_state_dict(torch.load(CHECKPOINT_PATH_TORCH + '%s-roberta-%i.bin'%(VER,i)))
    model.eval()
    
    model_list.append(model)


loading model 0
loading model 1
loading model 2
loading model 3
loading model 4


In [0]:
fold_ids = []
num_folds = 5

kfold = model_selection.KFold(n_splits=num_folds, shuffle=True, random_state=42)
for fold_num, (train_idx, valid_idx) in enumerate(kfold.split(train_df.text)):
    fold_ids.append((train_idx, valid_idx))

# DEV EVALUATION


In [0]:

with torch.no_grad():

    for fold_num in range(len(model_list)):
        print("Evaluation for Fold %i" % fold_num)
        final_output = []
        jacc_list = []
        
        valid_idx = fold_ids[fold_num][1]
        valid_df_fold = train_df.loc[valid_idx]

        valid_dataset = TweetDataset(
                tweet=valid_df_fold.text.values,
                sentiment=valid_df_fold.sentiment.values,
                selected_text=valid_df_fold.selected_text.values
            )

        data_loader = torch.utils.data.DataLoader(
            valid_dataset,
            shuffle=False,
            batch_size=config.VALID_BATCH_SIZE,
            num_workers=1
        )

        for d in tqdm(data_loader, total=len(data_loader), position=0, leave=True):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            sentiment = d["sentiment"]
            orig_selected = d["orig_selected"]
            orig_tweet = d["orig_tweet"]
            targets_start = d["targets_start"]
            targets_end = d["targets_end"]
            offsets = d["offsets"].numpy()

            ids = ids.to(config.DEVICE, dtype=torch.long)
            token_type_ids = token_type_ids.to(config.DEVICE, dtype=torch.long)
            mask = mask.to(config.DEVICE, dtype=torch.long)
            targets_start = targets_start.to(config.DEVICE, dtype=torch.long)
            targets_end = targets_end.to(config.DEVICE, dtype=torch.long)


    
            outputs_start, outputs_end = model_list[fold_num](
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids
            )

            outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
            outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
            jaccard_scores = []
            for px, tweet in enumerate(orig_tweet):
                selected_tweet = orig_selected[px]
                tweet_sentiment = sentiment[px]
                jacc, output_sentence = calculate_jaccard_score(
                    original_tweet=tweet,
                    target_string=selected_tweet,
                    sentiment_val=tweet_sentiment,
                    idx_start=np.argmax(outputs_start[px, :]),
                    idx_end=np.argmax(outputs_end[px, :]),
                    offsets=offsets[px]
                )
                final_output.append(output_sentence)
                jacc_list.append(jacc)
        train_df.loc[valid_idx,"pred"] = final_output
        train_df.loc[valid_idx,"jacc"] = jacc_list


 31%|███       | 213/687 [00:11<00:25, 18.40it/s]

# TEST PREDICTION

In [18]:
test_dataset = TweetDataset(
        tweet=df_test.text.values,
        sentiment=df_test.sentiment.values,
        selected_text=df_test.selected_text.values
    )

data_loader = torch.utils.data.DataLoader(
    test_dataset,
    shuffle=False,
    batch_size=config.VALID_BATCH_SIZE,
    num_workers=1
)

final_output = []
with torch.no_grad():
    for d in tqdm(data_loader, total=len(data_loader), position=0, leave=True):
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        sentiment = d["sentiment"]
        orig_selected = d["orig_selected"]
        orig_tweet = d["orig_tweet"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        offsets = d["offsets"].numpy()

        ids = ids.to(config.DEVICE, dtype=torch.long)
        token_type_ids = token_type_ids.to(config.DEVICE, dtype=torch.long)
        mask = mask.to(config.DEVICE, dtype=torch.long)
        targets_start = targets_start.to(config.DEVICE, dtype=torch.long)
        targets_end = targets_end.to(config.DEVICE, dtype=torch.long)

        for i in range(len(model_list)):
            outputs_start1, outputs_end1 = model_list[i](
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids
            )
            try:
                outputs_start += outputs_start1
                outputs_end += outputs_end1
            except:
                outputs_start = outputs_start1
                outputs_end = outputs_end1


            outputs_start /= 8
            outputs_end /= 8

        outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
        outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
        jaccard_scores = []
        for px, tweet in enumerate(orig_tweet):
            selected_tweet = orig_selected[px]
            tweet_sentiment = sentiment[px]
            _, output_sentence = calculate_jaccard_score(
                original_tweet=tweet,
                target_string=selected_tweet,
                sentiment_val=tweet_sentiment,
                idx_start=np.argmax(outputs_start[px, :]),
                idx_end=np.argmax(outputs_end[px, :]),
                offsets=offsets[px]
            )
            final_output.append(output_sentence)

100%|██████████| 442/442 [01:50<00:00,  4.00it/s]


In [0]:
df_test['selected_text'] = final_output

df_test['selected_text'] = df_test['selected_text'].apply(lambda x: x.replace('!!!!', '!') if len(x.split())==1 else x)
df_test['selected_text'] = df_test['selected_text'].apply(lambda x: x.replace('..', '.') if len(x.split())==1 else x)
df_test['selected_text'] = df_test['selected_text'].apply(lambda x: x.replace('...', '.') if len(x.split())==1 else x)

df_test[['textID','selected_text']].to_csv(DATA_PATH + 'submission-%s.csv'%VER, index=False)

In [20]:
df_test.head()

,textID,text,sentiment,selected_text
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,Last session of the day
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,exciting
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,such a shame!
3,01082688c6,happy bday!,positive,happy bday!
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,I like
